# TILT - Teemo Induced Loss of Tranquility
A research insentive to study main factors in inducing tilt.


# Imports

In [1]:
from collections import defaultdict
from pathlib2 import Path
from riotwatcher import LolWatcher, ApiError


# import arrow
import csv
import itertools
import datetime
import time
import json
import numpy as np
import pandas as pd

# Functions

In [2]:
  
def api_key(api_key_loc):
    """
    Read in the development API key from a file and checks if it is viable.
    
    Args:
        credentials (str): Name of json file containing the credentials.
    Returns:
        api_key (str): The API key.
    """

    while True:
        with open(api_key_loc, "r") as json_data:
            creds = json.load(json_data)
            api_key = creds["dev_api_key"]
            lol_watcher = LolWatcher(creds["dev_api_key"])
            try:
                # Validate API key by using it to check server status
                lol_watcher.lol_status.shard_data("euw1")
                # Break if key is functional
                break
            except ApiError as error:
                # If the current API key does not work input new one
                if error.response.status_code == 403:
                    new_api_key = input("API key is incorrect, enter correct key here.")
                    creds["dev_api_key"] = new_api_key
                    # Replace the old API key
                    with open(api_key_loc, "w") as json_data:
                        json.dump(creds, json_data)
    return api_key

In [3]:
def trans_reg(reg_abbrv):
    """
    Translate a league of legends region into a region readable by riot watcher.
    
    Args:
        reg_abbrv (str): Abbreviation of a official registered Riot servers that
                         hosts league of legends (e.g. euw1).
    Returns:
        rw_region (str): Riot Watcher region the reg_abbrv server falls under.
    """
    # Look up in the list what the riot watcher region is for the given region abbreviation
    regions_metadata = {"br1": "americas",
                        "eun1": "europe",
                        "euw1": "europe",
                        "jp1": "asia",
                        "kr": "asia",
                        "la1": "americas",
                        "la2": "americas",
                        "na1": "americas",
                        "oc1": "americas",
                        "tr1": "europe",
                        "ru": "europe"
                        }

    for reg, rw_reg in regions_metadata.items():
        if reg_abbrv.lower() == reg:
            rw_region = rw_reg
    
    return rw_region

In [4]:
def get_matches(reg, pid, s_time, e_time):
    """
    Retrieve match IDs from a player within a given timeframe.

    Args:
        reg (str): Abbreviation of a official registered Riot servers that
                   hosts league of legends (e.g. euw1).
        pid (str): An Encrypted globally unique identifyer for a player.
        s_time (long): Start of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
        e_time (long): end of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
    Returns:
        matches (list): League of legends match IDs in chronological order
                        starting with the most recent match.
    """

    # Retrieve all match ID's between two time points 
    matches = lol_watcher.match.matchlist_by_puuid(region=trans_reg(reg),
                                                   puuid=pid,
                                                   start_time=s_time,
                                                   end_time=e_time,
                                                   count=100)
    
    # Geather more match IDs in case matches exceeds the standard limit of 100
    while len(matches) % 100 == 0 and len(matches) != 0:
        # Geather match details of earliest match
        match_deets = lol_watcher.match.by_id(trans_reg(reg), matches[-1])
        # Start time of earliest match
        early_g_start = int(str(match_deets["info"]['gameCreation'])[:10])
        # Select 100 matches previous to early_g_start  
        match_add = lol_watcher.match.matchlist_by_puuid(region=trans_reg(reg),
                                                         puuid=pid,
                                                         start_time=s_time,
                                                         end_time=early_g_start,
                                                         count=100)
        if len(match_add) == 0:
            break
        else:
            matches.extend(match_add)

    return matches

In [96]:
def geather_data(puuid, reg, matches):
    """
    Geather player data from a match and chronology data of the match.
    
    Args:
        puuid (str): An Encrypted globally unique identifyer for a player.
        reg (str): Abbreviation of a official registered Riot servers that host league of legends (e.g. euw1).
        matches (list): League of legends match IDs in chronological order
                        starting with the most recent matches.  
    Returns:
        matches_data (df): Data geathered from all provided matches  
    """
    
    time_info = ["gameCreation", "gameStartTimestamp", "gameDuration"]
    keep_data = ["individualPosition", "role", "teamPosition", "lane", "kills", "assists", "deaths", 
                 "doubleKills", "tripleKills", "quadraKills", "pentaKills", 
                 "killingSprees", "largestKillingSpree", 
                 "teamEarlySurrendered", "gameEndedInEarlySurrender", "gameEndedInSurrender",
                 "neutralMinionsKilled", "totalMinionsKilled", "teamId", "win"]

    col_names = ["match_id", "game_make", "game_start", "game_dur", 
                "indiv_pos", "role", "team_pos", "lane", "kills", "assists", "deaths", 
                 "2_kills", "3_kills", "4_kills", "5_kills", 
                 "kill_spree", "max_kill_spree", 
                 "early_surr_try", "early_surr", "game_surr",
                 "neutral_kills", "mini_kills", "team_id", "win"]

    
    
    comp_data = []
    # Geather match details
    for match_id in matches:
        match_deets = lol_watcher.match.by_id(trans_reg(reg), match_id)        
        # Time data
        time_data = dict((key, match_deets["info"][key]) for key in time_info)
        # Player data       
        for part_info in match_deets["info"]["participants"]:
            if part_info["puuid"] == puuid:
                filt_data = dict((key, part_info[key]) for key in keep_data)
        match_data = {"match_id": match_id} | time_data | filt_data
        comp_data.append(match_data.values())
    
    # Store all data in a dataframe
    matches_data = pd.DataFrame(comp_data, columns=col_names)

    return matches_data
    

In [155]:
def calc_rest_time(matches_data):
    """
    Calculate the time in between matches.
    
    Args:
        reg (str): Abbreviation of a official registered Riot servers that host league of legends (e.g. euw1).
        matches (list): League of legends match IDs in chronological order
                        starting with the most recent matches.  
    Returns:
        rest_times (list): Determine from multiple matches the milliseconds between
                           the begin of champion select and the end of the previous match.
    """
    # time_info = matches_data[["game_make", "game_start", "game_dur"]]

    # move this to the filter block?
    matches_data["game_end"] = matches_data.loc[:,["game_start", "game_dur"]].sum(axis=1)
    matches_data["prev_game_make"] = matches_data["game_make"].shift(-1, fill_value=0)
    matches_data["rest_time"] = matches_data["game_end"] - matches_data["prev_game_make"]
    print(matches_data)

    # print(time_info)
    # print(time_info.loc[1:])
    # print(time_info[1:])
    # time_info = []
    # # Geather start and duration of each game
    # for match in matches:
    #     match_deets = lol_watcher.match.by_id(trans_reg(reg), match)
    #     time_info.append([match_deets["info"]["gameCreation"],
    #                       match_deets["info"]["gameStartTimestamp"],
    #                       match_deets["info"]["gameDuration"]])

    # rest_times = []
    # # Substract the end of the previous match from the current matches start
    # for match, prev_match in zip(time_info, time_info.loc[1:]):
    #     print(match, prev_match)
    #     # rest_times.append(match["gameCreation"] - (prev_match["gameStartTimestamp"] + prev_match["gameDuration"]))
    #     # rest_times.append(match[0] - (prev_match[1] + prev_match[2]))
    # # Oldest game has no previous match and thus a None is added
    # rest_times.append(None)
    
    # return rest_times

In [156]:
matches = ["EUN1_2923232971", "EUN1_2923257322", "EUN1_2923064617", "EUN1_2923101349"]
data = geather_data("WdfvM1gFWely8WNUtqV8ThlaO469Fu8D82jdDh4_4VRzdLqoFxVIEmU3crVeRjrYyPy0cY0c3CV8aw", "eun1", matches)
calc_rest_time(data)

          match_id      game_make     game_start  game_dur indiv_pos     role  \
0  EUN1_2923232971  1631108276000  1631108388636   1754772   UTILITY  SUPPORT   
1  EUN1_2923257322  1631104125000  1631104223486   1594271   UTILITY  SUPPORT   
2  EUN1_2923064617  1631100078000  1631100113446   1514201   UTILITY  SUPPORT   
3  EUN1_2923101349  1631097553000  1631097685770   1017875   UTILITY  SUPPORT   

  team_pos    lane  kills  assists  ...  early_surr_try  early_surr  \
0  UTILITY  BOTTOM      6       10  ...           False       False   
1  UTILITY  BOTTOM      1        7  ...           False       False   
2  UTILITY  BOTTOM      2        4  ...           False       False   
3  UTILITY    NONE      5        4  ...           False       False   

   game_surr  neutral_kills  mini_kills  team_id    win       game_end  \
0      False              4          48      200   True  1631110143408   
1      False              0          40      200  False  1631105817757   
2      False    

In [7]:
def filt_matches(reg, match_ids, max_rest, min_streak):
    """
    Filter matches based on rest time in between matches and number of matches played in a row.  
    
    Args:
        reg (str): Abbreviation of a official registered Riot servers that hosts league of legends (e.g. euw1).
        match_ids (list): League of legends match IDs in chronological order
                          starting with the most recent matches.  
        max_rest (int): Rest time in between matches in miliseconds e.g. 3600000 = 1 hour.
        min_streak (int): Minimum games played in a row with less then max_rest_time between   
    Returns:
        df_match_filt (df): Matches that ?????????????????
    """

    # TODO: reg can be read from match ID
    # Store the match IDs and the rest times in a dataframe
    df_match_filt = pd.DataFrame(list(zip(match_ids, calc_rest_time(reg, match_ids))), columns=["match_id", "rest_time"])

    # Filter for matches of interrest
    # Store if the game ID should be kept if rest time is below max_rest
    df_match_filt['keep'] = df_match_filt["rest_time"] <= max_rest
    # Grouping based on matches played subsequently 
    subseq_true = df_match_filt['keep'].diff().ne(0).cumsum()
    # Drop streaks that are lower then min_streak
    df_match_filt = df_match_filt[df_match_filt['keep'].groupby(subseq_true).transform("count") >= min_streak]
    # Drop matches with rest times above max_rest 
    df_match_filt = df_match_filt[~(df_match_filt["keep"] == 0.0)]

    # Add the streak id to the df_match_filt
    df_match_filt["streak_id"] = df_match_filt.groupby(subseq_true).grouper.group_info[0]

    # Remove the keep column 
    df_match_filt.drop("keep", axis=1, inplace = True)
    
    return df_match_filt


In [8]:

def get_summoners(regs, tiers, divs, sum_lim, s_time, e_time):
    """
    Get puuid's, tier and division, from all regions that are in ranks below master rank ((random?) above bronze?).
    
    Args:
        regs (list): Official registered Riot servers that hosts league of legends.
        tier (list): Tiers below master ranks.
        divs (list): Divisions in roman numerals.
        sum_lim (int): Maximum number of summoner ids to collect per region, tier and division
        s_time (long): Start of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
        e_time (long): end of a timeframe in (milli)seconds following
                       the Coordinated Universal Time (UTC) format.
    Returns:
        sumo_info (list): Summoners puuid, region, tier, and division.
    """
    
    summs_info=[]
    # for reg, tier, div in itertools.product(regs, tiers, divs): # real line change what we want in teh settings file
    for reg, tier, div in itertools.product(regs[0:1], tiers[1:2], divs[0:1]): # test line
        # print(reg, tier, div) # TODO: remove me later
        summs_div = 0
        page_nr = 0
        # Keep adding new summoners until the summoner limit has been reached
        while summs_div < sum_lim:
            page_nr += 1
            summs = lol_watcher.league.entries(reg, queue, tier, div, page_nr)
            # Look into data per summoner
            for sumo in summs:
                if summs_div < sum_lim:
                    # Get puuid
                    pid = lol_watcher.summoner.by_id(reg, sumo["summonerId"])["puuid"]
                    # Retrieve all match ID's between two time points
                    match_ids = get_matches(reg, pid, s_time, e_time)
                    
                    # TODO: could fit this into a try accept loop 
                    # Skip players that have no matches in the given time frame 
                    if len(match_ids) == 0:
                        continue
        
                    # GEATHER DATA FUNCTION
                    
                    # Filter matches
                    # TODO: these things you filter on should be in settings 
                    match_info = filt_matches(reg, match_ids, 3600000, 3)

                    # Store the info 
                    # TODO: Think of a smart dictionary/ list / dataframe solution for this
                    summs_info.append([pid, reg, tier, div, match_info]) # add time and change len to actual matches
                    summs_div += 1
            if summs_div == sum_lim:
                break

    return(summs_info)


In [9]:
# currently only used to check time
def convert_time(utc_time):
    """
    Converts UTC to GMT time.

    Args:
        time (long): Time in (milli)seconds following the Coordinated Universal Time (UTC) format.
    Returns:
        hr_time (str): Time in Greenwich Mean Time format (Year-Month-Day Hour:Minute:Second). 
    """

    time_in_secs = int(str(utc_time)[:10])
    gmt_time = str(datetime.datetime.fromtimestamp(time_in_secs))
    
    return(gmt_time)

# Main

In [10]:
# Directory locations

# Project folders
proj_dir = Path.cwd().parent

# Raw data storage
data_dir = proj_dir / "data"

# Out dir
out_dir = proj_dir / "out"

In [11]:
# Gobal variables

# Set API key
api_key_loc = data_dir / "dev_api_key.json"

# Enter API key
lol_watcher = LolWatcher(api_key(api_key_loc))

# Read user settings 
settings_loc = proj_dir / "settings" / "config.json"
with open(settings_loc, "r") as settings_data:
    settings = json.load(settings_data)
    # Min durtion of draft_pick in milliseconds, according to https://leagueoflegends.fandom.com/wiki/Draft_Pick 10-02-2022
    min_lobby_time = settings["min_lobby_time"]
    # Queue type of interrest
    queue = settings["queue"]
    # Official registered Riot servers of interrest
    regs = settings["regions"]
    # Tiers of interrest, below Master rank
    tiers = settings["tiers"]
    # Divisions of interrest, in roman numerals
    divs = settings['divisions']



    # things that could be removed if unused    
    # summoner name (not case sensitive)
    prota = settings["protagonist"]

### Collect summoners

In [13]:
# Retrieve the summoner info of X players
# move start end time to settings
s_time = 1609920000
e_time = 1631170800
summs_info = get_summoners(regs, tiers, divs, 1, s_time, e_time)

print(summs_info)


# Store the data in a dataframe
# sumo_info = pd.DataFrame(summs_info, columns=("puuid", "region", "tier", "division"))
# sumo_info

[['WdfvM1gFWely8WNUtqV8ThlaO469Fu8D82jdDh4_4VRzdLqoFxVIEmU3crVeRjrYyPy0cY0c3CV8aw', 'eun1', 'BRONZE', 'I',            match_id  rest_time  streak_id
2   EUN1_2923232971  2458243.0          0
3   EUN1_2923257322  2497353.0          0
4   EUN1_2923064617  1374355.0          0
5   EUN1_2923101349  3111249.0          0
8   EUN1_2894416345   167890.0          1
9   EUN1_2894412842   426965.0          1
10  EUN1_2894387447   264884.0          1
11  EUN1_2894352050   425159.0          1
26  EUN1_2883507364   296988.0          2
27  EUN1_2883413209  3047541.0          2
28  EUN1_2883370200  1033270.0          2
29  EUN1_2883312203   377635.0          2]]


### User info
Some simple scripts to retrieve basic information of the user not really needed just for quick test

In [5]:
convert_time(1644505520000)

'2022-02-10 16:05:20'

In [279]:
# retrieve info using region and summoner name # check region is wrong
user = lol_watcher.summoner.by_name('euw1', prota)
print(user)

# Check rank stats using user id
ranked_stats = lol_watcher.league.by_summoner("euw1", user["id"])
print(ranked_stats)

{'id': 'L2OqMzNqsCelFfcFlPx5iQWRB7dKLDdf9xGgO_W1dNau3EI', 'accountId': 'fYybzHGf_HSjunX7VEs-9jXIEvVLk-eQhAyhq_VzaXWi2w', 'puuid': 'tKPszy2ygrLuiS7AYcfh4pjDzc3jQO2xOIjAf4YKq8XjytTznNaQc88wPYMmJkONT1JjtkioHEfaMg', 'name': 'Alnas', 'profileIconId': 3382, 'revisionDate': 1617835118000, 'summonerLevel': 109}


### 2/3 Get match list by PUUID's

In [368]:
# Create a dictionary to store all the results

for row in sumo_info.itertuples():
    for match_id in row.match_ids: # match id
        match_deets = lol_watcher.match.by_id(row.riot_watcher_region, match_id)
        for k, v in match_deets.items():
             print(k, '>', v)
        # print(match_deets['info'])
        # for k in match_deets['info']:
        #     print(k)
        # things of interrest     
        # print(type(match_deets))


#         game_id = match["gameId"]
#         queue_type = match["queue"]
#         match_detail = lol_watcher.match.by_id(region, game_id)
#         match_start = match["timestamp"]
#         match_duration = match_detail["gameDuration"]*1000
#         lobby_end = match_detail["gameCreation"]

#         # Store filtered information pertinent to the match itself
#         match_info[game_id]["game_info"]["game_type"] = queue_type
#         match_info[game_id]["game_info"]["begin_time"] = lobby_end - min_lobby_time
#         match_info[game_id]["game_info"]["end_time"] = match_start + match_duration

        break
    # Collect match details


metadata > {'dataVersion': '2', 'matchId': 'EUW1_5722779994', 'participants': ['_yHPg3ee266u0jfpkDiiHINIIB-HvIpxyIMSM7qOquBSbxaf_Kr_yPtyj-Y76s711dud8023BMSADw', 'QSh7HumTtVrR4cVtD-qNdDm7slUZMeoMvhXdQQWeDI5hqfb2Iz15UGkvXtENrM6G18ag-s_KKZ2-iA', 'jcb62REEFDAZUxRCWVlq5IhuKmPkIHymoouFiMdwoluUVIe0xJAFosqysXKIKcTeoza2e6cdQ6ig_A', 'KG0c7frvFoXbq73AKfwm28xubKeJiMZwRIjc0WNSHoWWYy-401kgzTV_tDre2CYBdWTR6VjfDj4AqQ', 'c_L5dvu5BCO_QTtWNTJ_41ZUETB8IX52FegHPbQl-VpPnSHWxTHmhj7uPkicqhUDjinctACbRdB6xQ', 'rbDNeqIIavsLjSLdVaD6Iz1ktmEWt9X1BxTjP7hRuJ30Qn6gc1hfpUx8oBts-izKotQQ1L0wm_W8tA', '1oTRWtiyNtZENiagrcbo7ijGMlIb0IgcmhcFHIzIi-aneWSCVk8FSNsdtOa-8SOOtz7zD_UPTLMjbQ', 'dtYUu4WRg9v0nnvaZjkhcLueQpCHAkLvN4PQOnJSv6vGANbfKbccI7bDtqMsBBpjPmGrM5Xh1WWffg', 'PkyDyFsJQ35GQYruyFBfALnEvWKhRNSPqNyq15L6wgmY4WWH_ddk1ufUMtNh2-ioNtJvHwo-aY3c-w', 'M8agh_QgrDjQ02Ulf1NcFr-f1osn-HMxuaLnamJusYu0NLI-ZGRj8tmaWmaOPTbqjEo69XCzNBzu5w']}
info > {'gameCreation': 1644505520000, 'gameDuration': 1908, 'gameEndTimestamp': 1644507548617, 'gam

In [ ]:

match_info = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
# for match in matches["matches"][:1]:
#         # Collect information pertinent to the match itself
#         game_id = match["gameId"]
#         queue_type = match["queue"]
#         match_detail = lol_watcher.match.by_id(region, game_id)
#         match_start = match["timestamp"]
#         match_duration = match_detail["gameDuration"]*1000
#         lobby_end = match_detail["gameCreation"]

#         # Store filtered information pertinent to the match itself
#         match_info[game_id]["game_info"]["game_type"] = queue_type
#         match_info[game_id]["game_info"]["begin_time"] = lobby_end - min_lobby_time
#         match_info[game_id]["game_info"]["end_time"] = match_start + match_duration

#         # Store filtered information pertinent to participants
#         protagonist_blue = None
#         # Collect participant ID and summoner name
#         for participant in match_detail["participantIdentities"]:
#                 part_id = participant["participantId"]
#                 summoner_name = participant["player"]["summonerName"]
#                 # Storing summoner name using participant ID as key
#                 match_info[game_id]["player_data"][part_id]["participant_id"] = part_id 
#                 match_info[game_id]["player_data"][part_id]["name"] = summoner_name
#                 if summoner_name.lower() == protagonist.lower():
#                         protagonist_blue = True if part_id <=5 else False

#         # Collect kills, deaths, assists and win
#         for participant in match_detail["participants"]:
#                 part_id = participant["stats"]["participantId"]
#                 kills = participant["stats"]["kills"] 
#                 deaths = participant["stats"]["deaths"]
#                 assists = participant["stats"]["assists"]
#                 kda = round((kills+assists)/ max(1, deaths), 3)
#                 win = participant["stats"]["win"]

#                 # Store kills, deaths, assists, kda and win
#                 match_info[game_id]["player_data"][part_id]["kills"] = kills
#                 match_info[game_id]["player_data"][part_id]["deaths"] = deaths
#                 match_info[game_id]["player_data"][part_id]["assists"] = assists
#                 match_info[game_id]["player_data"][part_id]["kda"] = kda
#                 match_info[game_id]["player_data"][part_id]["win"] = win

#                 # Assign participants as ally or enemy of the protagonist                   
#                 match_info[game_id]["player_data"][part_id]["allied"] = bool(part_id <=5) == protagonist_blue

# print("done", datetime.datetime.now())
# # TODO: print how long it took

In [ ]:
# short script to IO files

summ_list_loc = out_dir / "summoner_list.tsv"
with open(summ_list_loc, "w") as out_file:    
    sumo_info.to_csv(out_file, sep='\t', index = False, line_terminator='\n')


# Read in the summoner information
sumo_info = pd.read_csv(summ_list_loc, sep = "\t")


In [23]:
# Output for for easy reading
for i in match_info.keys():
        print("this is match", i)
        print(match_info[i].keys())
        print(match_info[i]["game_info"].keys())
        print(convert_time(match_info[i]["game_info"]["begin_time"]))
        print(convert_time(match_info[i]["game_info"]["end_time"]))
        print("")

        print("match info")
        for y in match_info[i]["game_info"]:
                print(match_info[i]["game_info"][y])

        print("")
        print("player info")
        for player in match_info[i]["player_data"]:
                print(match_info[i]["player_data"][player])

        print("\n------------------------\n")

In [ ]:
# todo: write a function that measures how long a block took to run
# datetime.now()

In [71]:
with open(data_folder / "patches.json") as f:
    data = json.load(f)

patch = data["patches"][-1]  # most recent
utc_timestamp = patch["start"]

north_america_timestamp = utc_timestamp + data["shifts"][region.upper()]
print(north_america_timestamp)
dt = arrow.get(north_america_timestamp)
print(dt.to("US/Pacific"))

1607493600
2020-12-08T22:00:00-08:00


In [ ]:

# participants = []
# for row in match_detail['participants']:
#     participants_row = {}
#     participants_row['champion'] = row['championId']
#     # participants_row['spell1'] = row['spell1Id']
#     # participants_row['spell2'] = row['spell2Id']
#     participants_row['win'] = row['stats']['win']
#     participants_row['kills'] = row['stats']['kills']
#     participants_row['deaths'] = row['stats']['deaths']
#     participants_row['assists'] = row['stats']['assists']
#     participants_row['KDA'] = kda(row['stats']['kills'], row['stats']['deaths'], row['stats']['assists'])
#     participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
#     participants_row['goldEarned'] = row['stats']['goldEarned']
#     participants_row['champLevel'] = row['stats']['champLevel']
#     participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
#     # participants_row['item0'] = row['stats']['item0']
#     # participants_row['item1'] = row['stats']['item1']
#     participants.append(participants_row)
# df = pd.DataFrame(participants) # this step is still important 07-2
# # df

In [44]:

thingy = np.arange(220,551,110)

print(thingy)

[220 330 440 550]


### Patch an season info

In [62]:
# example of how to Read time
# 2018-01-24T03:00:00-08:00  (for patch 8.2) --> corresponds to 3am on January 24th, 2018 in Pacific Time

with open(data_folder / "patches.json") as f:
    data = json.load(f)

patch = data["patches"][-1]  # most recent
utc_timestamp = patch["start"]
north_america_timestamp = utc_timestamp + data["shifts"]["NA1"]
print(north_america_timestamp)
dt = arrow.get(north_america_timestamp)
print(dt.to("US/Pacific"))



1607515200
2020-12-09T04:00:00-08:00


In [ ]:
# Not needed anymore 
def kda(kill, death, assist):
    """
    Calculate kill assist to death ratio.
    If a player would have 0 deaths, a value of 1 will be used to avoid a KDA of infinity.
    
    Args:
        kill (int): Number of kills performed by the player.
        death (int): Number of deaths of the player.
        assists (int): Number of times the player has assisted another player in a kill.
    Returns:
        kda_score (int): The number of kills and/or assists a player got per death on average.
    """

    kda_score = round((kill+assist) / max(1, death), 3)

    return kda_score

In [ ]:
# example end script for wayyy later

try:
    response = lol_watcher.summoner.by_name(region, summoner_name)
    print(response)
except ApiError as error:
    if error.response.status_code == 429:
        print('Request limit reached retry in {} seconds.'.format(err.headers['Retry-After']))
    elif error.response.status_code == 404: 
        print("Found no match")

In [ ]:
# try except loop
    #     try:
    #         # Retrieve the matches between two time points
    #         match_list = lol_watcher.match.matchlist_by_puuid(reg,
    #                                                           row.puuid,
    #                                                           start_time=s_time,
    #                                                           end_time=e_time,
    #                                                           count=100)     
    #     except ApiError as error:
    #         if error.response.status_code == 400:
    #             continue
